<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">pyCRYPTO</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);"><b style=color:red;>Crypto</b>graphy</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
<td><img style="height: 150px;" src="images/pyCRYPTO.png"></td>
</tr>
</table>

----
# `pyCRYPTO`

pyCRYPTO, a program package for cryptography tools and blockchains.

# Blockchain
----

In this notebook, we derive a simple **blockchain** algorithm.

[Idea from](https://www.geeksforgeeks.org/create-simple-blockchain-using-python/)

In [1]:
import hashlib,datetime,json

----
# Blockchain
A **blockchain** is a list of data blocks, which are linked together across neighboring blocks with a cryptographic hash function.
This ensures that changes in the data structure of the blockchain can be detected.

Typical elements of a **blockchain** are:
- hash value of previous block
- time stamp
- transaction (e.g. a message, ...)
- ...

To initialize a blockchain, usually a **genesis block** is used as first block.

All other blocks are generated by calculation a **hash value** from the previous block, then,
upon creation, solving a task, often called **proof of work**, and then the new block is
added to the blockchain.

The **proof of work** can be a complicated exercise, e.g. a **solution of a differential equation**,
or a **problem from crypography**.

Next, we assemble the different parts...

----
## Set up blockchain
The first function creates an empty blockchain as list.

In [2]:
def blockchain_init():
    """
    blockchain:
    function initializes a blockchain
    input:
      (none)
    output:
      chain - the empty blockchain dictionary (as list)
    """
    # create empty list for chain
    chain = [] #dict()
    return chain

----
## Genesis block
The second function fills the first block in the blockchain, the so-called **genesis block**.
Here, the structure of the blockcain ist defined, in our case:
- `index:` The index number, starting at 1.
- `timestamp:` the current date and time.
- `data:` the message part of the block.
- `proof:` the calczulated value for the **proof of work**, set to `1` in the genesis block (could be any number).
- `previous_hash:` The hash value for the previous block, set to `0`in the genesis block (could be any string).

In [3]:
def blockchain_genesisBlock(chain):
    """
    blockchain:
    function adds a genesis block
    input:
      chain - empty blockchain
    output:
      chain - the  blockchain with genesis block
    """
    # define genesis block as first block
    genesisBlock = {
            'index': 1,
            'timestamp': str(datetime.datetime.now()),
            'data': "Genesis block of blockchain",
            'proof': 1,
            'previous_hash': '0'}
    chain.append(genesisBlock)
    return chain

Test the two functions:

In [19]:
myChain = blockchain_init()
myChain = blockchain_genesisBlock(myChain)
print(type(myChain))
for i in range(len(myChain)):
    print(i,myChain[i])

<class 'list'>
0 {'index': 1, 'timestamp': '2025-01-26 19:47:33.738553', 'data': 'Genesis block of blockchain', 'proof': 1, 'previous_hash': '0'}


----
## Technical functions
We define some technical functions:
- The function `blockchain_previousBlock`, which returns the entire last bock from the blockchain.
- The function ``,

In [5]:
def blockchain_previousBlock(chain):
    """
    blockchain:
    function returns the last block in the blockchain
    """
    return chain[-1]

In [6]:
blockchain_previousBlock(myChain)

{'index': 1,
 'timestamp': '2025-01-26 19:37:04.423684',
 'data': 'Genesis block of blockchain',
 'proof': 1,
 'previous_hash': '0'}

In [7]:
def blockchain_hash(block):
    """
    blockchain:
    function returns the sha256 hash value for the data of an entire block
    """
    encoded_block = json.dumps(block, sort_keys=True).encode()
    return hashlib.sha256(encoded_block).hexdigest()

In [8]:
blockchain_hash(myChain)

'f4237076960699f355a4ca93f40be5f4d37e5fb2871a704c447a2c9b932340c7'

----
## News blocks
Next, we create a new potential block for the blockchain.
We fill the fields, especially 
- the `data` part, the message for this block.
- the `previous_hash`, calculated e.g. as `sha256` hash of the entire last block.
- the `proof` as solution of the proof of work exercise, used as starting value for the next block.

In [9]:
def blockchain_createBlock(chain,data='',proof=1,previous_hash='0'):
    """
    blockchain:
    function to create a new block
    """
    block = {
            'index': len(chain) + 1,
            'timestamp': str(datetime.datetime.now()),
            'data': str(data),
            'proof': proof,
            'previous_hash': previous_hash}
    chain.append(block)
    return chain

In [10]:
myChain = blockchain_createBlock(myChain)
for i in range(len(myChain)):
    print(i,myChain[i])

0 {'index': 1, 'timestamp': '2025-01-26 19:37:04.423684', 'data': 'Genesis block of blockchain', 'proof': 1, 'previous_hash': '0'}
1 {'index': 2, 'timestamp': '2025-01-26 19:37:20.343730', 'data': '', 'proof': 1, 'previous_hash': '0'}


----
## Proof of work
The proof of work is usually a function of calculation, which takes some time and is used to be repetible for
later comparison, e.g. the solution of an initial-value problem from a given start value.

We use here a classical example from crytography:
- start with a given number `new_proof`
- use the result from the last block `previous_proof`
- loop, in which a hash value of an operation ($new\_proof^2 - previous\_proof^2$) is calculated
- The loop is exited, if the first five characters of the hash value are `00000`

We run this as example loop:

In [11]:
previous_proof = 1
new_proof = 1
check_proof = False

i=0
while check_proof is False:
    hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
    i += 1
    if hash_operation[:5] == '00000':
        check_proof = True
    else:
        new_proof += 1
print(i,hash_operation,previous_proof,new_proof)

632238 00000d27548b48fb9948dec841504bb2dfe0ad4812f0f6c049f2cd02dada6dcd 1 632238


The loop took $632238$ iterations, until the hast function fulfils the criterion.

Moved to a function, it reads:

In [12]:
def blockchain_proofOfWork(previous_proof):
    """
    blockchain:
    function for proof of work, used to mine a new block
    """
    # start counter
    new_proof = 1
    check_proof = False
    # increase counter, until first five digits of hash function are zero 
    while check_proof is False:
        hash_operation = hashlib.sha256(
            str(new_proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:5] == '00000':
            check_proof = True
        else:
            new_proof += 1
    return new_proof

In [13]:
newProof = blockchain_proofOfWork(1)
print(newProof)

632238


----
## Mining new blocks
(text)

In [14]:
def blockchain_mineBlock(chain):
    """
    blockchain:
    function mines a new block
    input: 
    - chain
    output
    - updated chain
    
    """
    previous_block = blockchain_previousBlock(chain)
    previous_proof = previous_block['proof']
    proof = blockchain_proofOfWork(previous_proof)
    previous_hash = blockchain_hash(previous_block)
    data = 'A block is MINED'
    chain = blockchain_createBlock(chain,data,proof,previous_hash)
    return chain

In [22]:
def blockchain_addBlock(chain):
    previous_block = blockchain_previousBlock(chain)
    previous_proof = previous_block['proof']
    proof = blockchain_proofOfWork(previous_proof)
    previous_hash = blockchain_hash(previous_block)
    data = 'A block is ADDED'
    chain = blockchain_createBlock(chain,data,proof,previous_hash)
    return chain

In [20]:
myChain = blockchain_mineBlock(myChain)

In [24]:
myChain = blockchain_addBlock(myChain)

In [25]:
for i in range(len(myChain)):
    print(myChain[i])

{'index': 1, 'timestamp': '2025-01-26 19:47:33.738553', 'data': 'Genesis block of blockchain', 'proof': 1, 'previous_hash': '0'}
{'index': 2, 'timestamp': '2025-01-26 19:47:52.354977', 'data': 'A block is MINED', 'proof': 632238, 'previous_hash': 'a9c6b47cde831069dd726fc9b1b890bbdf012383f179994cc559ac982d75ca34'}
{'index': 3, 'timestamp': '2025-01-26 19:47:59.083763', 'data': 'A block is ADDED', 'proof': 403091, 'previous_hash': '7f85df2f26961a9dee8385b58eb0ef0d72747f3c29c00d0aeda7c7663c70ab1b'}
{'index': 4, 'timestamp': '2025-01-26 19:48:03.577862', 'data': 'A block is ADDED', 'proof': 714736, 'previous_hash': '348eb5e07b6de755e9bb182a6ea1804c2230483d0dcee972d34501e50c33c4d1'}


In [17]:
def blockcain_chainValid(chain):
    """
    blockchain:
    functions tests the validity of the entire blockchain in two steps:
    1. Check if the previous_hash value is the one from the previous block
    2. ...
    """
    # start with first block of blockchain
    previous_block = chain[0]
    block_index = 1
    # run loop over all blocks
    while block_index < len(chain):
        block = chain[block_index]
        # check if previous block has been manipulated by 
        # calculating and comparing hash value
        if block['previous_hash'] != blockchain_hash(previous_block):
            return False
        # check proof of work value
        previous_proof = previous_block['proof']
        proof = block['proof']
        hash_operation = hashlib.sha256(
            str(proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:5] != '00000':
            return False
        previous_block = block
        block_index += 1

    return True

In [26]:
valid = blockcain_chainValid(myChain)
if valid:
    print('The Blockchain is valid.')
else:
    print('The Blockchain is not valid.')

The Blockchain is valid.


----